### Dataset

CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classe

### Goal 

building a Classification model using Transfer learning using ResNet50 as base model. 

In [1]:

import numpy as np
import random
import numpy as np
import matplotlib.pyplot as plt
import os
from six.moves import cPickle as pickle
from sklearn import svm

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import scipy

import tensorflow as tf


from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,UpSampling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D,BatchNormalization
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import cv2

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train.shape

(50000, 32, 32, 3)

In [4]:
X_train= X_train/255
X_test = X_test/255

In [5]:
#One-hot encoding the labels
num_classes = 10

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [6]:
#Loading the ResNet50 model with pre-trained ImageNet weights
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [7]:

model1 = Sequential()
model1.add(UpSampling2D((2,2)))
model1.add(UpSampling2D((2,2)))
model1.add(UpSampling2D((2,2)))
model1.add(resnet)
model1.add(Flatten())
model1.add(BatchNormalization())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dense(10, activation='softmax'))


In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=2e-5)
model1.compile(loss='categorical_crossentropy', optimizer=opt, 
              metrics=['accuracy'])

#history = model.fit(x_train, y_train, epochs=5, batch_size=20, validation_data=(x_test, y_test))


In [10]:
checkpointer =ModelCheckpoint(filepath='resnet_second.best.hdf5',
                               verbose=1,save_best_only=True)


model1.fit(X_train, Y_train,epochs=5, batch_size=32, 
          validation_split=0.2)

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1250/1250 [==============================] - 6449s 5s/step - loss: 1.6471 - accuracy: 0.4558 - val_loss: 0.8202 - val_accuracy: 0.7811
Epoch 2/5
1250/1250 [==============================] - 6452s 5s/step - loss: 1.0219 - accuracy: 0.6892 - val_loss: 0.5022 - val_accuracy: 0.8784
Epoch 3/5
1250/1250 [==============================] - 6456s 5s/step - loss: 0.7640 - accuracy: 0.7947 - val_loss: 0.3962 - val_accuracy: 0.9068
Epoch 4/5
1250/1250 [==============================] - 6458s 5s/step - lo

In [11]:
model1.save('model_resnet_second.h5')

In [13]:
score  = model1.evaluate(X_test, Y_test)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

313/313 [==============================] - 311s 993ms/step - loss: 0.2958 - accuracy: 0.9249
Accuracy on the Test Images:  0.9248999953269958
